In [5]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)


def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()


def get_comment(user_id):
    q = query(f"""select comment.content
    from comment
    join user
    on user.id = comment.author_id
    where comment.uuid_id = user.id and user.id = {user_id}""")
    
    return q[0][0] if len(q) > 0 else ""
    
    
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df["comment_on_own_page"] = df["id"].map(get_comment)  
df_potential_bot = df[(df["edits"] + df["reviews"] + df["taxonomy"] == 0) & (df["comment_on_own_page"] != "")][["id", "username", "comment_on_own_page"]].copy()
df_potential_bot.sort_values(["id"], inplace=True)
df_potential_bot.to_csv("/tmp/2022-21-03users_with_comments_on_own_page.csv")
df_potential_bot


,id,username,comment_on_own_page
2057,41334,anjanajain727,\r\nNon commercial relocation is definitely an...
2892,59143,n_dersim62,"Ich finde es total doof, dass es keine Themen ..."
2990,61630,amitsulu,Relocation and boring job seems synonyms when ...
5592,90995,ShikhaVerma,Really impressive post. I read it whole and go...
7894,115228,prekhshasharma,Get Packers and Movers Kolkata List of Top Rel...
8893,121905,truewickedsick,https://en.serlo.org
11025,137246,Rolex2014,Gib es irgendwann vielleicht auch eine Serlo-A...
11151,138941,69Madde69Pladde69,ok
13034,164603,1234567,Ich komme mit dem neuen Editor noch nicht so ...
13716,176558,solutionss,"<a href=""\r\nhttps://shieldsecuritysolutions.c..."
